In [1]:
import os
import shutil
import tensorflow
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image

MUSHROOMS_PATH = 'mushrooms_dataset'

# Directory for the images and its subdirectories
images_dir = os.path.join(MUSHROOMS_PATH, 'images')
subdirs = [os.path.join(images_dir, subdir) for subdir in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, subdir))]

In [7]:
#Now we have some ideas for dividing the dataset into training and testing sets. We can use the train_test_split function from scikit-learn to divide the dataset into training and testing sets.
#But for that we will have to put the every image into array and then into a dataframe
#Then we will have to use ImageDataGenerator and flow_from_dataframe to load the images from the dataframe

#Second idea is to manually create the test set by taking 20% of the images from each class and putting them into a separate directory.
#We will then use ImageDataGenerator and flow_from_directory to load the images from the directory.

#In both ideas we need to take in consider stratification, so that the distribution of classes in the training and testing sets is similar.
#For example, if in one class there are 10 images and in another one there are 8 images, we want both  of them to have the same percentage of images in the training and testing sets.

#Third idea is to use the splitfolders library to divide the dataset into training and testing sets.
#But again we have to stratify the dataset which is not supported by that library.

#So the first idea might require a lot of memory usage, the second idea needs us to well do this manually which is not very efficient.
#And the third idea is not supporting stratification.

#So for now we will use the first idea and divide the dataset into training and testing sets using the train_test_split function from scikit-learn which has the stratify parameter.


In [8]:
#So the process with the first idea is as follows:
#1. Load the images and its corresponding labels into a dataframe.
#2. Divide the dataset into training and testing sets using the train_test_split function from scikit-learn with stratification.
#3. Use ImageDataGenerator and flow_from_dataframe to load the images from the dataframe.

In [2]:
data = []
for subdir in subdirs:
    label = os.path.basename(subdir) #we specify the label for each image
    for filename in os.listdir(subdir):
        if filename.endswith('.jpg'):
            data.append((os.path.join(subdir, filename), label)) #we need to include whole path of the image for using flow_from_dataframe because it reads the images directly from the file system using the paths provided in the DataFrame.
data_df = pd.DataFrame(data, columns=['filename', 'label'])

In [3]:
data_df.head()

,filename,label
0,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis
1,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis
2,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis
3,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis
4,mushrooms_dataset\images\Abortiporus_biennis\1...,Abortiporus_biennis


In [4]:
train_df, test_df = train_test_split(data_df, test_size=0.2, stratify=data_df['label'], random_state=42)

In [5]:
datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, shear_range=0.2, validation_split=0.25,) #we use 25% from the 80% of the training set as the validation set which will be the same amount as the testing set

train_data = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=64,
    subset='training'
)


val_data = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=64,
    subset='validation'
)

datagen_test = ImageDataGenerator(rescale=1./255)

test_data = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='label',
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=64
)

Found 120015 validated image filenames belonging to 3976 classes.
Found 40005 validated image filenames belonging to 3976 classes.
Found 40005 validated image filenames belonging to 3976 classes.


In [6]:
#---DONE---
#It worked, but why do we have only 6903 classes in the test set and 7504 in training and validation sets? 
# Perhaps there are not enough images in some classes???

# It is probably true beacause when we use stratify parameter in train_test_split function, it tries to keep the distribution of classes in the training and testing sets similar.
# But if there are not enough images in some classes, it will not be able to keep the distribution of classes similar in the training and testing sets.
# So we have few solutions to this
# 1. Ensure that each class has a minimum number of instances before splitting the data into training and testing sets - that worked!!!!
# 2. Use the stratify sampling only on the classes with sufficient instances, and randomly split the ones with too few instances

Could have - we could create our own model but since we have massive amount of images its easier to use pre-trained one

In [ ]:
# Creating the first model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(7504, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(train_data, validation_data=val_data, epochs=20)



In [ ]:
# model.evaluate(test_data)

Próba treningu na pretrenowanym modelu InceptionV3

In [7]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
predictions = Dense(3976, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, validation_data=val_data, epochs=10)




Epoch 1/10


1876/1876 [==============================] - 744s 395ms/step - loss: 7.2602 - accuracy: 0.0244 - val_loss: 6.9447 - val_accuracy: 0.0301
Epoch 2/10
1876/1876 [==============================] - 530s 282ms/step - loss: 6.7019 - accuracy: 0.0348 - val_loss: 6.7207 - val_accuracy: 0.0339
Epoch 3/10
1876/1876 [==============================] - 530s 283ms/step - loss: 6.4736 - accuracy: 0.0381 - val_loss: 6.6073 - val_accuracy: 0.0386
Epoch 4/10
1876/1876 [==============================] - 526s 280ms/step - loss: 6.3484 - accuracy: 0.0407 - val_loss: 6.5646 - val_accuracy: 0.0412
Epoch 5/10
1876/1876 [==============================] - 527s 281ms/step - loss: 6.2764 - accuracy: 0.0425 - val_loss: 6.5708 - val_accuracy: 0.0408
Epoch 6/10
1876/1876 [==============================] - 746s 398ms/step - loss: 6.2299 - accuracy: 0.0442 - val_loss: 6.5625 - val_accuracy: 0.0410
Epoch 7/10
1876/1876 [==============================] - 1231s 656ms/step - loss: 6.2064 - accuracy: 0.0452 

In [8]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_1
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [9]:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [10]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10
1876/1876 [==============================] - 1502s 800ms/step - loss: 6.0247 - accuracy: 0.0504 - val_loss: 6.1153 - val_accuracy: 0.0603
Epoch 2/10
1876/1876 [==============================] - 1475s 786ms/step - loss: 5.6559 - accuracy: 0.0713 - val_loss: 6.0000 - val_accuracy: 0.0693
Epoch 3/10
1876/1876 [==============================] - 1523s 812ms/step - loss: 5.4556 - accuracy: 0.0854 - val_loss: 5.8874 - val_accuracy: 0.0807
Epoch 4/10
1876/1876 [==============================] - 1465s 781ms/step - loss: 5.2901 - accuracy: 0.0985 - val_loss: 5.8149 - val_accuracy: 0.0892
Epoch 5/10
1876/1876 [==============================] - 1348s 718ms/step - loss: 5.1454 - accuracy: 0.1115 - val_loss: 5.7604 - val_accuracy: 0.1006
Epoch 6/10
1876/1876 [==============================] - 669s 357ms/step - loss: 4.9925 - accuracy: 0.1244 - val_loss: 5.7603 - val_accuracy: 0.1075
Epoch 7/10
1876/1876 [==============================] - 653s 348ms/step - loss: 4.8548 - accuracy: 0.1382 -

In [12]:
#Evaluate the model
model.evaluate(test_data)

626/626 [==============================] - 133s 212ms/step - loss: 5.4644 - accuracy: 0.1455


[5.464415073394775, 0.1455318033695221]